In [4]:
!pip3 install opencv-python

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import pandas as pd
import numpy as np

from glob import glob

import cv2 as cv
import matplotlib.pyplot as plt
import os

In [3]:

cap = cv.VideoCapture(0)

if not cap.isOpened():
    print ("Error: Could not open the camera.")
    exit()

input_directory = 'InputStream'
os.makedirs(input_directory, exist_ok=True)
max_frames = 60
frame_counter = 0

while True:
    frames = [f for f in os.listdir(input_directory) if f.endswith('.jpg')]
    ret, frame = cap.read()

    frame_filename = os.path.join(input_directory, f"frame_{frame_counter}.jpg")

    cv.imwrite(frame_filename, frame)
    frame_counter += 1
    if len(frames) >= max_frames:
        os.remove(f'InputStream/frame_{frame_counter - max_frames}.jpg')
    
    cv.imshow('Video Stream', frame)

    if cv.waitKey(1) == ord('q'):
        break


cap.release()
cv.destroyAllWindows()



In [ ]:
file = [items for items in os.listdir(input_directory) if items.endswith('.jpg')]
for k in file:
    file_path = os.path.join(input_directory, k)
    os.remove(file_path)

: 